In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
data_path = '../data/pdb_2021aug02_sample'

import glob

glob.glob(data_path + '/*')

[]

In [4]:
import pandas as pd

pd.read_csv(f'{data_path}/list.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/pdb_2021aug02_sample/list.csv'

In [9]:
import csv
from dateutil import parser

def build_training_clusters(params, debug):
    val_ids = set([int(l) for l in open(params['VAL']).readlines()])
    test_ids = set([int(l) for l in open(params['TEST']).readlines()])

    if debug:
        val_ids = []
        test_ids = []

    # read & clean list.csv
    with open(params['LIST'], 'r') as f:
        reader = csv.reader(f)
        next(reader)
        rows = [[r[0],r[3],int(r[4])] for r in reader
                if float(r[2])<=params['RESCUT'] and
                parser.parse(r[1])<=parser.parse(params['DATCUT'])]

    # compile training and validation sets
    train = {}
    valid = {}
    test = {}

    if debug:
        rows = rows[:20]
    for r in rows:
        if r[2] in val_ids:
            if r[2] in valid.keys():
                valid[r[2]].append(r[:2])
            else:
                valid[r[2]] = [r[:2]]
        elif r[2] in test_ids:
            if r[2] in test.keys():
                test[r[2]].append(r[:2])
            else:
                test[r[2]] = [r[:2]]
        else:
            if r[2] in train.keys():
                train[r[2]].append(r[:2])
            else:
                train[r[2]] = [r[:2]]
    if debug:
        valid=train
    return train, valid, test



data_path = '../russell_protein/protein_mpnn/tests/fixtures/pdb_2021aug02_sample'

train,valid,test = build_training_clusters({
    'VAL': f'{data_path}/valid_clusters.txt',
    'TEST': f'{data_path}/test_clusters.txt',
    'LIST': f'{data_path}/list.csv',
    'RESCUT': 2.0,
    'DATCUT': '2021-01-01'

}, True)

In [41]:
train

{6750: [['5nag_A', '082310'],
  ['5nag_B', '082310'],
  ['5nah_A', '082311'],
  ['5nah_B', '082311'],
  ['5nak_A', '082310'],
  ['5nak_B', '082310']],
 21385: [['5nai_A', '067953']],
 22021: [['5naj_A', '030830'],
  ['5naj_B', '030830'],
  ['5naj_C', '030830'],
  ['5naj_D', '030830']],
 15132: [['5nar_A', '036425'], ['5nat_A', '036425'], ['5naw_A', '036425']],
 1421: [['5nay_A', '103501'],
  ['5nay_B', '103501'],
  ['5nay_C', '103501'],
  ['5nay_D', '103501'],
  ['5nay_E', '103501'],
  ['5nay_F', '103501']]}

In [ ]:
item = train[6750][3]

pdbid, chid = item[0].split('_')
PREFIX = "%s/pdb/%s/%s"%('DIR',pdbid[1:3],pdbid)
pdbid, PREFIX

('5nah', 'DIR/pdb/na/5nah')

In [ ]:
import torch
def load_chain_data(pdb_id, chain_id):
    path = f'{data_path}/pdb/l3/{pdb_id}_{chain_id}.pt'
    return torch.load(path)

pdb_id = '1l3a'
d = load_chain_data('6l3t', 'B')

/var/folders/95/40g56m011614lbl69wt_5llc0000gn/T/ipykernel_14584/579976422.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path)


In [75]:

assembly_chains


['A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R']

In [126]:
meta

{'method': 'X-RAY_DIFFRACTION',
 'date': '2002-02-26',
 'resolution': 2.65,
 'chains': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
 'seq': [['MIPDDEFIKNPSVPGPTAMEVRCLIMCLAEPGKNDVAVDVGCGTGGVTLELAGRVRRVYAIDRNPEAISTTEMNLQRHGLGDNVTLMEGDAPEALCKIPDIDIAVVGGSGGELQEILRIIKDKLKPGGRIIVTAILLETKFEAMECLRDLGFDVNITELNIARGRALDRGTMMVSRNPVALIYTGVSHENKD',
   'MIPDDEFIKNPSVPGPTAMEVRCLIMCLAEPGKNDVAVDVGCGTGGVTLELAGRVRRVYAIDRNPEAISTTEMNLQRHGLGDNVTLMEGDAPEALCKIPDIDIAVVGGSGGELQEILRIIKDKLKPGGRIIVTAILLETKFEAMECLRDLGFDVNITELNIARGRALDRGTMMVSRNPVALIYTGV------'],
  ['MIPDDEFIKNPSVPGPTAMEVRCLIMCLAEPGKNDVAVDVGCGTGGVTLELAGRVRRVYAIDRNPEAISTTEMNLQRHGLGDNVTLMEGDAPEALCKIPDIDIAVVGGSGGELQEILRIIKDKLKPGGRIIVTAILLETKFEAMECLRDLGFDVNITELNIARGRALDRGTMMVSRNPVALIYTGVSHENKD',
   'MIPDDEFIKNPSVPGPTAMEVRCLIMCLAEPGKNDVAVDVGCGTGGVTLELAGRVRRVYAIDRNPEAISTTEMNLQRHGLGDNVTLMEGDAPEALCKIPDIDIAVVGGSGGELQEILRIIKDKLKPGGRIIVTAILLETKFEAMECLRDLGFDVNITELNIARGRALDRGTMMVSRNPVALIYTGV------'],
  ['MIPDDEFIKNPSVPGPTAMEVRCLIMCLAEPGKNDVAVDVGCGTGGVTLELAGRVRRVYAID

In [ ]:

cached_chain_data = {}
def load_chain_data(pdb_id, chain_id):

    key = f'{pdb_id}_{chain_id}'
    if key in cached_chain_data:
        return cached_chain_data[key]

    path = f'{data_path}/pdb/l3/{pdb_id}_{chain_id}.pt'
    data = torch.load(path)

    cached_chain_data[key] = data
    return data


pdb_id = '1l3b'
structure_metadata_path = f'{data_path}/pdb/l3/{pdb_id}.pt'

meta = torch.load(structure_metadata_path)
assembly_ids = meta['asmb_ids']
assembly_chains = meta['asmb_chains']
chain_ids = meta['chains']

chain_id = 'A'

for assembly_index, asembly_id in enumerate(assembly_ids):
    chains = []
    assembly_chains = assembly_chains[assembly_index].split(',')
    for c in assembly_chains:
        if c in meta['chains']:
            chains.append(c)

    if chain_id not in chains:
        continue

    print(asembly_id)
    print(chains)

    # asmb_xformIDX  - (one per biounit) xforms to be applied to chains from asmb_chains[IDX], [n,4,4]
    # [n,:3,:3] - rotation matrices
    # [n,3,:3] - translation vectors
    xform = meta[f'asmb_xform{assembly_index}']
    rotation_matrix_u = xform[:,:3,:3]
    translation_matrix_r = xform[:,:3,3]

    chain_data = {}
    for chain in chains:
        print(chain)
        chain_details = load_chain_data(pdb_id, chain)
        chain_data[chain] = chain_details

        xyz = chain_details['xyz']
        xyz_ru = torch.einsum('bij,raj->brai', rotation_matrix_u, xyz) + translation_matrix_r[:,None,None,:]

        print(xyz)


1
['A', 'D', 'E', 'H']
A
tensor([[[ 21.6800, -16.4960, 158.8530],
         [ 21.5690, -15.1810, 159.5550],
         [ 21.2360, -15.3720, 161.0360],
         ...,
         [     nan,      nan,      nan],
         [     nan,      nan,      nan],
         [     nan,      nan,      nan]],

        [[ 21.9540, -14.6640, 161.9010],
         [ 21.7230, -14.7760, 163.3330],
         [ 20.2810, -14.4710, 163.7110],
         ...,
         [     nan,      nan,      nan],
         [     nan,      nan,      nan],
         [     nan,      nan,      nan]],

        [[ 19.5770, -15.4610, 164.2760],
         [ 18.1870, -15.2170, 164.6650],
         [ 18.0800, -14.2410, 165.8450],
         ...,
         [     nan,      nan,      nan],
         [     nan,      nan,      nan],
         [     nan,      nan,      nan]],

        ...,

        [[     nan,      nan,      nan],
         [     nan,      nan,      nan],
         [     nan,      nan,      nan],
         ...,
         [     nan,      nan,      nan

/var/folders/95/40g56m011614lbl69wt_5llc0000gn/T/ipykernel_14584/2925260667.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load(structure_metadata_path)
/

In [104]:
chids = np.array(meta['chains'])
chids[chids == 'A']

array(['A'], dtype='<U1')

In [ ]:
seqid = meta['tm'][chids=='A'][0,:,1]
seqid

homo = set([
    ch_j
    for seqid_j,ch_j in zip(seqid,chids)
    if seqid_j>params['HOMO']
])

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
seqid = meta['tm'][chids==chid][0,:,1]
homo = set([
    seqid_j, ch_j for seqid_j,ch_j in zip(seqid,chids)
    if seqid_j>.70 #params['HOMO']
])
homo

{(tensor(1.), np.str_('A')),
 (tensor(1.), np.str_('B')),
 (tensor(1.), np.str_('C')),
 (tensor(1.), np.str_('D')),
 (tensor(1.), np.str_('E')),
 (tensor(1.), np.str_('F')),
 (tensor(1.), np.str_('G')),
 (tensor(1.), np.str_('H'))}

In [6]:
import sys
sys.path.append('../')

from russell_protein.protein_mpnn.dataset import load_assembly_from_pdb_id_containing_chain

In [12]:
data_path = '../russell_protein/protein_mpnn/tests/fixtures/pdb_2021aug02_sample'
data_path

'../russell_protein/protein_mpnn/tests/fixtures/pdb_2021aug02_sample'

In [1]:
asm = load_assembly_from_pdb_id_containing_chain(
    '1l3a',
    'A',
    data_path,
    .90
)

NameError: name 'load_assembly_from_pdb_id_containing_chain' is not defined

In [34]:
asm[0].keys()

dict_keys(['seq', 'seq_list', 'xyz', 'idx', 'masked', 'label'])

In [38]:
[
    len(x) for x in asm[0]['seq_list']
]

[227, 227, 227, 227]

In [48]:
asm[0]['masked']

tensor([0, 1, 2, 3], dtype=torch.int32)

In [144]:
meta['tm'][chids=='A'][0,:,1]

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

In [148]:
meta['tm'][5,:,1]

tensor([1., 1., 1., 1., 1., 1., 1., 1.])